# Purpose

In the last few notebooks, we created a variety of functions that allow us to generate billings for our supermarket for any amount of time.

In this notebook, we will do it for 1 year.

In [1]:
import numpy as np
import pandas as pd
import uuid
import random
from datetime import timedelta

In [2]:
# Functions
def generate_bill(customer: pd.Series, items: pd.DataFrame) -> (list, list):
    """
    Purchases for a single customer.
    
    We add items to a customer's basket, at random, and check if the
    sub-total exceeds the customer's budget.
    """
    items_purchased = []
    sub_total = []
    cust_budget = customer['budget']
    while sum(sub_total) < cust_budget:
        item = items.sample()
        sub_total.append(item['price'].values[0])
        items_purchased.append(item['uuid'].values[0])
    if sum(sub_total) > customer['budget']:
        sub_total.pop()
        items_purchased.pop()
    return items_purchased, sub_total


def generate_time() -> tuple:
    """Generates a random time between 8am and 10pm, when our store is open."""
    rtime   = int(random.random() * 86_400)
    
    hours   = rtime//3600
    minutes = (rtime - hours*3600)//60
    seconds = rtime - hours*3600 - minutes*60

    if (hours <= 8) or (hours >= 22):
        return generate_time()
    else:
        return hours, minutes, seconds


def generate_timestamp(date):
    """Returns a UNIX timestamp for a given date + random time."""
    return int((date + timedelta(*generate_time())).timestamp())

In [3]:
# Loading data
customers = pd.read_csv('../data/processed/customers_database.csv')
display(customers.sample(5))

items = pd.read_csv('../data/processed/items_database.csv')
display(items.sample(5))

,age,habit,budget,uuid
54,29,weekly,26.5,2983884575
46,52,weekly,26.5,2546376173
25,61,weekly,26.5,2630012468
94,48,weekly,26.5,3391121701
17,92,weekly,26.5,3745847835


,product,price,type,uuid
182,Whiskas 7+ Fish Jelly Pouch Senior Cat Food 12...,3.29,pets,60791
43,Warburtons Fruit Teacakes 4 Pack,0.90,bakery,55757
177,Felix Cat Food Fish Selection In Jelly 12 X 100G,3.65,pets,46636
8,Clementine Or Sweet Easy Peeler Pack 600G,1.35,fresh-food,26820
169,Felix Cat Food Mixed Selection In Jelly 40X100g,11.00,pets,54369


In the previous notebook, we made it so we generated a date in which weekly customers would come to store. For simplicity, we will assume that sporadic customers will come to store in a random day once a month.

To facilitate our processing, let's divide the `customers` dataset into 3, based on the shopping habits of customers.

In [4]:
daily = customers.query('habit == "daily"')
weekly = customers.query('habit == "weekly"')
sporadic = customers.query('habit == "sporadic"')

Let's generate our billings!

In [66]:
%%time
billings = []

# We'll define an initial week number and month number
init_week = 0
init_month = 0

for day in pd.date_range(start='2031-01-01', end='2031-12-31'):
    
    this_week = day.week
    if this_week != init_week:
        # We generate the day of visit for weekly customers
        weekly['day_in'] = np.random.randint(
            0, 7, 
            size=len(weekly)
        )
        # And make it so the week we check against is the same
        init_week = this_week
    
    this_month = day.month
    if this_month != init_month:
        # And we do the same for the sporadic customers
        sporadic['day_in'] = np.random.randint(
            0, 
            31, 
            size=len(sporadic)
        )
        init_month = this_month
    
    # we get the weekday from the timestamp
    todays_customers = daily.append(
        weekly.query(f'day_in == {int(day.strftime("%w"))}')
    ).append(
        sporadic.query(f'day_in == {day.day}')
    )
    
    for idx, customer in todays_customers.set_index('uuid').iterrows():
        bill = generate_bill(customer, items)
        bill_id = uuid.uuid4().fields[0]
        ts = generate_timestamp(day)
        for it,pr in zip(bill[0], bill[1]):
            billings.append(
                {
                    'id': bill_id,
                    'timestamp': ts,
                    'customer_id': idx,
                    'items': it,
                    'total': pr
                }
            )

<timed exec>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 12.3 s, sys: 97.1 ms, total: 12.4 s
Wall time: 12.4 s


It only took a few seconds to generate 1 years worth of billings for 100 customers. Technology is amazing. Let's have a look at the data we generated.

In [67]:
billings_df = pd.DataFrame(billings)
display(billings_df.head(10))

,id,timestamp,customer_id,items,total
0,859724875,1926288046,958420868,36088,0.75
1,859724875,1926288046,958420868,7244,0.79
2,2768611031,1926288019,4036240255,5651,1.48
3,2768611031,1926288019,4036240255,5049,0.50
4,2768611031,1926288019,4036240255,64051,1.20
5,688223310,1926633614,3031153441,53480,0.75
6,688223310,1926633614,3031153441,15694,5.50
7,688223310,1926633614,3031153441,25737,1.50
8,688223310,1926633614,3031153441,13573,2.50
9,688223310,1926633614,3031153441,57769,1.00


In [21]:
# Let's save the data for posterity
billings_df.to_csv('../data/processed/billings.csv', index=False)

### Saving to SQL database

The final step before moving on (which, admittedly, is an optional one), is to store the data into a database for later querying. We also need to store the items into databases, so we may know which items were bought (at the very least, we need the product type and id).

We will start by importing sqlite3, since we just want to create a local database.

In [25]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """
    Create a database connection to a SQLite database.
    
    The db_file should be the path to a .db file.
    
    If you pass the file name as :memory: to the connect() function 
    of the sqlite3 module, it will create a new database that resides 
    in the memory (RAM) instead of a database file on disk.
    
    Returns a connection object.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    
    except Error as e:
        print(e)

    return conn

Next, we create a function that creates a table based on a SQL query. We also need a query.

In [26]:
def create_table(conn, create_table_sql):
    """
    Create a table from the create_table_sql statement.
    
    Requires a valid SQL connection.
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

Let's try to create a table for the items. The syntax to create a table is

```
CREATE TABLE IF NOT EXISTS table_name (
	col1_name data_type PRIMARY KEY,
	col2_name data_type NOT NULL,
);
```

For us, the query becomes

```
CREATE TABLE IF NOT EXISTS items (
    id integer PRIMARY KEY,
    product_name text NOT NULL,
    type text NOT NULL,
    price float NOT NULL
);
```

In [70]:
# We create a path to the database
database = "../databases/groceries.db"

create_items_table = """CREATE TABLE IF NOT EXISTS items (
                            id integer PRIMARY KEY,
                            product_name text NOT NULL,
                            type text NOT NULL,
                            price float NOT NULL
                        );"""

# create a database connection
conn = create_connection(database)

# create table
if conn is not None:
    # create items table
    create_table(conn, create_items_table)

else:
    print("Cannot create the database connection.")

And we insert the values into the database, rowwise.

In [71]:
def insert(conn, item):
    """
    Create a new item into the items table
    """
    sql = '''INSERT INTO items(id,product_name,type,price)
              VALUES(?,?,?,?)'''
    cur = conn.cursor()
    cur.execute(sql, item)
    conn.commit()
    return cur.lastrowid

In [72]:
for idx,row in items[['uuid', 'product', 'type', 'price']].iterrows():
    insert(conn, tuple(row.tolist()))

Let's check our new table.

In [73]:
cursor = conn.cursor()
for row in cursor.execute('select * from items;'):
    print(row)
conn.close()

(32, 'Homestyle Straight Cut Oven Chips 950G', 'frozen-food', 1.4)
(552, 'Pampers Baby Dry Pants Essential Pack Size 6 28 Nappies', 'baby', 7.0)
(885, 'Fred & Flo 60 Sensitive Toilet Wipes', 'baby', 0.7000000000000001)
(1584, 'Colgate Total Original Care Toothpaste 125Ml', 'health-and-beauty', 2.5)
(1614, "Carte D'or Vanilla Ice Cream Dessert 1L", 'frozen-food', 3.5)
(1750, 'Comfort Pure Fabric Conditioner 85 Wash 3L', 'baby', 4.0)
(1960, 'Dove Intensive Repair Shampoo 400Ml', 'health-and-beauty', 1.79)
(2172, 'Pampers Baby Dry Pants Essential Pack Size 5 33 Nappies', 'baby', 7.0)
(2648, "Mcvitie's Jaffa Cakes Triple Pack 30 Cakes", 'food-cupboard', 2.0)
(2846, 'Luxury Soft Toilet Tissue 9 Roll White', 'household', 2.95)
(3058, 'Huggies Pure Baby Wipes Fragrance Free Sngle Pack 56 Wipes', 'baby', 0.9)
(3410, 'Luxury Soft Toilet Tissue 4 Roll White', 'household', 1.0)
(3431, "Antiseptic Wipes 10'S", 'baby', 0.95)
(3628, 'Kingsmill 50/50 Medium Bread 800G', 'bakery', 0.85)
(3776, "Dog Po

We got it! Now we need to do the same for the billings database.

In [74]:
create_billings_table = """CREATE TABLE IF NOT EXISTS billings (
                            id integer NOT NULL,
                            timestamp integer NOT NULL,
                            customer_id interger NOT NULL,
                            item integer NOT NULL
                        );"""

# create a database connection
conn = create_connection(database)

# create table
if conn is not None:
    # create items table
    create_table(conn, create_billings_table)

In [75]:
def insert_into_billings(conn, billing):
    """
    Create a new billing into the billings table
    """
    sql = '''INSERT INTO billings(id,timestamp,customer_id,item)
              VALUES(?,?,?,?)'''
    cur = conn.cursor()
    cur.execute(sql, billing)
    conn.commit()

for idx,row in billings_df.drop('total', axis=1).iterrows():
    insert_into_billings(conn, tuple(row.tolist()))

In [76]:
cursor = conn.cursor()
for row in cursor.execute('select * from billings;'):
    print(row)
conn.close()

(859724875, 1926288046, 958420868, 36088)
(859724875, 1926288046, 958420868, 7244)
(2768611031, 1926288019, 4036240255, 5651)
(2768611031, 1926288019, 4036240255, 5049)
(2768611031, 1926288019, 4036240255, 64051)
(688223310, 1926633614, 3031153441, 53480)
(688223310, 1926633614, 3031153441, 15694)
(688223310, 1926633614, 3031153441, 25737)
(688223310, 1926633614, 3031153441, 13573)
(688223310, 1926633614, 3031153441, 57769)
(688223310, 1926633614, 3031153441, 8560)
(688223310, 1926633614, 3031153441, 19394)
(55789631, 1925856027, 17985943, 19394)
(55789631, 1925856027, 17985943, 33533)
(55789631, 1925856027, 17985943, 17407)
(55789631, 1925856027, 17985943, 57741)
(55789631, 1925856027, 17985943, 14679)
(55789631, 1925856027, 17985943, 16744)
(55789631, 1925856027, 17985943, 64051)
(55789631, 1925856027, 17985943, 41231)
(55789631, 1925856027, 17985943, 62913)
(55789631, 1925856027, 17985943, 49522)
(55789631, 1925856027, 17985943, 57741)
(55789631, 1925856027, 17985943, 52171)
(557896

(1467166255, 1927411253, 3956634413, 15921)
(1467166255, 1927411253, 3956634413, 48274)
(1467166255, 1927411253, 3956634413, 34797)
(1467166255, 1927411253, 3956634413, 17670)
(1467166255, 1927411253, 3956634413, 64782)
(1467166255, 1927411253, 3956634413, 14117)
(1467166255, 1927411253, 3956634413, 41765)
(1467166255, 1927411253, 3956634413, 27893)
(1467166255, 1927411253, 3956634413, 49258)
(1467166255, 1927411253, 3956634413, 30026)
(1467166255, 1927411253, 3956634413, 25592)
(1467166255, 1927411253, 3956634413, 52022)
(1467166255, 1927411253, 3956634413, 45061)
(1467166255, 1927411253, 3956634413, 6954)
(1467166255, 1927411253, 3956634413, 43391)
(1467166255, 1927411253, 3956634413, 35254)
(1467166255, 1927411253, 3956634413, 25983)
(1467166255, 1927411253, 3956634413, 25764)
(1467166255, 1927411253, 3956634413, 15921)
(1467166255, 1927411253, 3956634413, 31803)
(1467166255, 1927411253, 3956634413, 2648)
(1467166255, 1927411253, 3956634413, 1960)
(1467166255, 1927411253, 3956634413

(2333624999, 1928275216, 1720291968, 49742)
(2333624999, 1928275216, 1720291968, 43875)
(2333624999, 1928275216, 1720291968, 12612)
(2333624999, 1928275216, 1720291968, 51583)
(2333624999, 1928275216, 1720291968, 33067)
(2189934784, 1928016045, 958420868, 31614)
(2616492663, 1928102402, 4036240255, 1584)
(2616492663, 1928102402, 4036240255, 33464)
(7153567, 1927843204, 269896857, 25070)
(1995806432, 1928620851, 3031153441, 18940)
(1995806432, 1928620851, 3031153441, 26820)
(1995806432, 1928620851, 3031153441, 13222)
(1995806432, 1928620851, 3031153441, 6133)
(1995806432, 1928620851, 3031153441, 17670)
(1995806432, 1928620851, 3031153441, 32)
(1995806432, 1928620851, 3031153441, 65180)
(1995806432, 1928620851, 3031153441, 26820)
(1995806432, 1928620851, 3031153441, 41069)
(1995806432, 1928620851, 3031153441, 4367)
(1995806432, 1928620851, 3031153441, 41371)
(1995806432, 1928620851, 3031153441, 885)
(3246675250, 1928707225, 3745847835, 28474)
(3246675250, 1928707225, 3745847835, 57741)
(

(3235097644, 1929312051, 2588521857, 32306)
(3235097644, 1929312051, 2588521857, 49522)
(3235097644, 1929312051, 2588521857, 23587)
(13568464, 1929398427, 958420868, 1960)
(13568464, 1929398427, 958420868, 3776)
(2220588332, 1929484827, 4036240255, 32306)
(2220588332, 1929484827, 4036240255, 54600)
(2224283317, 1929398451, 269896857, 17407)
(255644729, 1929139219, 3982346988, 56792)
(255644729, 1929139219, 3982346988, 11921)
(255644729, 1929139219, 3982346988, 41181)
(255644729, 1929139219, 3982346988, 41371)
(255644729, 1929139219, 3982346988, 49315)
(255644729, 1929139219, 3982346988, 36126)
(255644729, 1929139219, 3982346988, 59580)
(255644729, 1929139219, 3982346988, 29284)
(255644729, 1929139219, 3982346988, 8560)
(255644729, 1929139219, 3982346988, 37445)
(2409879642, 1929916800, 17985943, 54442)
(2409879642, 1929916800, 17985943, 8309)
(2409879642, 1929916800, 17985943, 46366)
(2409879642, 1929916800, 17985943, 57458)
(2409879642, 1929916800, 17985943, 29991)
(2409879642, 192991

(35290074, 1930694422, 2326982145, 43094)
(35290074, 1930694422, 2326982145, 25070)
(35290074, 1930694422, 2326982145, 16846)
(35290074, 1930694422, 2326982145, 57741)
(35290074, 1930694422, 2326982145, 44662)
(35290074, 1930694422, 2326982145, 13484)
(35290074, 1930694422, 2326982145, 55439)
(35290074, 1930694422, 2326982145, 3058)
(35290074, 1930694422, 2326982145, 48607)
(35290074, 1930694422, 2326982145, 20301)
(35290074, 1930694422, 2326982145, 60756)
(3003588727, 1930521645, 3397042353, 23479)
(3003588727, 1930521645, 3397042353, 64051)
(3003588727, 1930521645, 3397042353, 56792)
(3003588727, 1930521645, 3397042353, 35566)
(3003588727, 1930521645, 3397042353, 20944)
(3003588727, 1930521645, 3397042353, 26352)
(3003588727, 1930521645, 3397042353, 50576)
(3003588727, 1930521645, 3397042353, 33464)
(3003588727, 1930521645, 3397042353, 41218)
(3003588727, 1930521645, 3397042353, 27787)
(3003588727, 1930521645, 3397042353, 57458)
(3003588727, 1930521645, 3397042353, 7969)
(3003588727,

...

(2985045355, 1954108800, 2656631279, 2846)
(2985045355, 1954108800, 2656631279, 20944)
(2985045355, 1954108800, 2656631279, 24958)
(2985045355, 1954108800, 2656631279, 52196)
(2985045355, 1954108800, 2656631279, 54600)
(2985045355, 1954108800, 2656631279, 17910)
(2985045355, 1954108800, 2656631279, 36088)
(2985045355, 1954108800, 2656631279, 23479)
(2985045355, 1954108800, 2656631279, 25502)
(2985045355, 1954108800, 2656631279, 24737)
(2985045355, 1954108800, 2656631279, 62726)
(2985045355, 1954108800, 2656631279, 42932)
(2985045355, 1954108800, 2656631279, 65180)
(2985045355, 1954108800, 2656631279, 31614)
(2985045355, 1954108800, 2656631279, 33067)
(2985045355, 1954108800, 2656631279, 41231)
(2985045355, 1954108800, 2656631279, 29445)
(2985045355, 1954108800, 2656631279, 63618)
(2985045355, 1954108800, 2656631279, 2172)
(2985045355, 1954108800, 2656631279, 17914)
(2985045355, 1954108800, 2656631279, 11726)
(2985045355, 1954108800, 2656631279, 49315)
(2985045355, 1954108800, 265663127

(2008451426, 1954368050, 1377453970, 52269)
(2008451426, 1954368050, 1377453970, 57741)
(2008451426, 1954368050, 1377453970, 33067)
(2008451426, 1954368050, 1377453970, 28028)
(2008451426, 1954368050, 1377453970, 54698)
(2008451426, 1954368050, 1377453970, 14679)
(2008451426, 1954368050, 1377453970, 53480)
(2008451426, 1954368050, 1377453970, 24056)
(2008451426, 1954368050, 1377453970, 52196)
(2008451426, 1954368050, 1377453970, 17914)
(2008451426, 1954368050, 1377453970, 52381)
(2008451426, 1954368050, 1377453970, 17407)
(2008451426, 1954368050, 1377453970, 63381)
(2008451426, 1954368050, 1377453970, 25737)
(2008451426, 1954368050, 1377453970, 25592)
(2988911588, 1954886416, 4036240255, 30026)
(2988911588, 1954886416, 4036240255, 13573)
(958912794, 1954972827, 269896857, 35319)
(3333402925, 1954540847, 3785220396, 39686)
(3333402925, 1954540847, 3785220396, 39193)
(3333402925, 1954540847, 3785220396, 13484)
(3333402925, 1954540847, 3785220396, 14679)
(3333402925, 1954540847, 378522039

(105785041, 1956009656, 4079302491, 53728)
(105785041, 1956009656, 4079302491, 43094)
(105785041, 1956009656, 4079302491, 11426)
(105785041, 1956009656, 4079302491, 49742)
(1258965983, 1955664046, 4008714709, 26335)
(1258965983, 1955664046, 4008714709, 57769)
(1258965983, 1955664046, 4008714709, 29284)
(1258965983, 1955664046, 4008714709, 35254)
(1258965983, 1955664046, 4008714709, 17914)
(1258965983, 1955664046, 4008714709, 18940)
(1258965983, 1955664046, 4008714709, 39962)
(1258965983, 1955664046, 4008714709, 12612)
(1258965983, 1955664046, 4008714709, 12612)
(3050506515, 1955232057, 2546376173, 33464)
(3050506515, 1955232057, 2546376173, 25592)
(3050506515, 1955232057, 2546376173, 64782)
(3050506515, 1955232057, 2546376173, 35254)
(3050506515, 1955232057, 2546376173, 25592)
(3050506515, 1955232057, 2546376173, 2846)
(3050506515, 1955232057, 2546376173, 17707)
(3050506515, 1955232057, 2546376173, 23056)
(3050506515, 1955232057, 2546376173, 18736)
(3050506515, 1955232057, 2546376173, 

(3769676873, 1956441648, 1604618946, 61107)
(3769676873, 1956441648, 1604618946, 48607)
(3769676873, 1956441648, 1604618946, 25592)
(3769676873, 1956441648, 1604618946, 11836)
(3769676873, 1956441648, 1604618946, 41765)
(3769676873, 1956441648, 1604618946, 25592)
(3769676873, 1956441648, 1604618946, 54369)
(3769676873, 1956441648, 1604618946, 1584)
(3769676873, 1956441648, 1604618946, 33533)
(1689193274, 1956355256, 209495655, 21511)
(1689193274, 1956355256, 209495655, 46225)
(1689193274, 1956355256, 209495655, 64049)
(1689193274, 1956355256, 209495655, 11836)
(1689193274, 1956355256, 209495655, 64049)
(1689193274, 1956355256, 209495655, 45061)
(1689193274, 1956355256, 209495655, 11726)
(1689193274, 1956355256, 209495655, 1960)
(1689193274, 1956355256, 209495655, 14588)
(1689193274, 1956355256, 209495655, 43875)
(1689193274, 1956355256, 209495655, 15921)
(1689193274, 1956355256, 209495655, 43391)
(1689193274, 1956355256, 209495655, 21511)
(1689193274, 1956355256, 209495655, 17910)
(168

(3052011327, 1957737654, 3785220396, 49742)
(3052011327, 1957737654, 3785220396, 7244)
(3052011327, 1957737654, 3785220396, 35254)
(3052011327, 1957737654, 3785220396, 57769)
(3052011327, 1957737654, 3785220396, 38204)
(3052011327, 1957737654, 3785220396, 57769)
(3052011327, 1957737654, 3785220396, 50576)
(3052011327, 1957737654, 3785220396, 4992)
(3052011327, 1957737654, 3785220396, 53728)
(3463041575, 1957651243, 2630012468, 23876)
(3463041575, 1957651243, 2630012468, 43391)
(3463041575, 1957651243, 2630012468, 14117)
(3463041575, 1957651243, 2630012468, 49742)
(3463041575, 1957651243, 2630012468, 12388)
(3463041575, 1957651243, 2630012468, 13484)
(3463041575, 1957651243, 2630012468, 11426)
(3463041575, 1957651243, 2630012468, 57525)
(3463041575, 1957651243, 2630012468, 2846)
(3463041575, 1957651243, 2630012468, 46225)
(3463041575, 1957651243, 2630012468, 51583)
(3463041575, 1957651243, 2630012468, 19902)
(3463041575, 1957651243, 2630012468, 36427)
(3463041575, 1957651243, 2630012468

It's beautiful. Now we can do some analysis on the random data we generated... in the next notebook!